In [2]:
import pandas as pd

# create csv file to use for test

# dict = {
#     "Weight": [7, 4, 15, 15, 15, 15, 15, 15, 15, 15, 15, 25, 24, 24, 24, 24, 25],
#     "Grade": ["10/10", "100/100", "5/5", "4.68/5", "5/5", "5/5", "5/5", "5/5", "5/5", "4.9/4.9", "5/5", "80/100", "180/200", "180/200", "173/200", "200/200", "90/100"],
#     "SectionName": ["Discussion", "Participation", "Worksheets", "Worksheets", "Worksheets", "Worksheets", "Worksheets", "Worksheets", "Worksheets", "Worksheets", "Worksheets", "Exams", "SA Writing", "SA Writing", "SA Writing", "SA Writing", "Final Exam"]
# }
# gradesDF = pd.DataFrame(dict)
# gradesDF
# gradesDF.to_csv("Grades.csv", index=False)

In [3]:
def createDictionary(gradesDF):
    dict = {}
    for column in gradesDF:
        dict[column] = gradesDF[column].tolist()
    return dict

In [4]:
def canStringBeNum(strings):
    canBeNum = True
    validNumbers = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    decimalRecorded = False
    
    for string in strings:
        if canBeNum == False:
            break
        for char in string:
            if canBeNum == False:
                break
            if not char in validNumbers:
                if char == "." and len(string) > 1:
                    decimalRecorded = True
            elif not char in validNumbers:
                canBeNum = False

    return canBeNum

In [7]:
def calculate(grades, gradesDF):
    totalWeight = 0
    listLength = 0
    totalScoresList = []
    sectionScores = []
    sectionMaxScores = []
    recipientsGrades = {}

    def findColumn(listOfColumnNames):
        columnFound = "N/A"
        for columnString in listOfColumnNames:
            if columnFound != "N/A":
                break
            for key in grades.keys():
                newKey = key.lower().strip()
                if columnString == newKey:
                    gradesDF.rename(columns = {key: columnString}, inplace = True)
                    columnFound = columnString
                    break
        return columnFound

    weightColumnName = findColumn(["weight"])
    scoreColumnName = findColumn(["grade", "score", "point"])
    sectionColumnName = findColumn(["section", "sectionname"])

    if scoreColumnName != "N/A":
        failedAttempts = 0
        maxFailedAttempts = len(grades.keys()) + 3
        if sectionColumnName != "N/A":
            weight = 1 # default so it doesn't affect score if no weight column
            group = gradesDF.groupby(sectionColumnName)
            recordedSections = []

            for sec in gradesDF[sectionColumnName].tolist():
                if not sec in recordedSections:
                    recordedSections.append(sec)
                    groupList = group.get_group(sec)[scoreColumnName].tolist()
                    if listLength < len(groupList):
                        listLength = len(groupList)

            recordedSections = []
            for section in gradesDF[sectionColumnName].tolist():
                if not section in recordedSections:
                    recordedSections.append(section)
                    maxScoreForWeight = 0
                    scoreForWeight = 0
                    listOfScores = []
                    groupList = group.get_group(section)[scoreColumnName].tolist()
                    if weightColumnName != "N/A":
                        weight = group.get_group(section)[weightColumnName].tolist()[0] # doesn't matter, weight is same as section
                        totalWeight = totalWeight + weight
                    for score in groupList:
                        if isinstance(score, str):
                            if "/" in score:
                                if canStringBeNum(score.split("/")) == True:
                                    num1 = float(score.split("/")[0].strip())
                                    num2 = float(score.split("/")[1].strip())
                                    listOfScores.append(str(score) + " | " + str(round((num1 / num2) * 100, 2)) + "%")
                                    maxScoreForWeight = maxScoreForWeight + num2
                                    scoreForWeight = scoreForWeight + num1
                                else:
                                    return "error: make sure you enter the points you earned and point total - Example: 2/3"
                            else:
                                return "error: make sure you split your score and the total score with '/' - Example: 2/3"
                        else:
                            return "error: make sure scores are in string format"
        
                    scoreEarnedForSection = round((scoreForWeight / maxScoreForWeight) * 100, 2)
                    listOfScores.append(section + " Score: " + str(scoreEarnedForSection) + "%")
                    if len(groupList) != listLength:
                        difference = listLength - len(groupList)
                        for x in range(difference):
                            listOfScores.insert(0, "NaN")
                    if weightColumnName != "N/A":
                        recipientsGrades[section + " (Weight: " + str(weight) + "%):"] = listOfScores
                        sectionScores.append(scoreEarnedForSection * (weight / 100))
                    else:
                        recipientsGrades[section + ":"] = listOfScores
                        sectionScores.append(scoreForWeight)
                        sectionMaxScores.append(maxScoreForWeight)
        else:
            return "error: section names can not be found, please add a section column with the names of each section the grade is in."
    else:
        return "error: scores can not be found, please add a score column with all of your grades."
                    
   
    for i in range(listLength):
        totalScoresList.append("NaN")

    scoreEarned = 0
    maxScoreCanReceive = 0
    if weightColumnName != "N/A":
        for sectionScore in sectionScores:
            scoreEarned = scoreEarned + (sectionScore / 100)
        totalScoresList.append("Total Score: " + str(round(scoreEarned * 100, 2)) + "%")
        recipientsGrades["Total (Weight: " + str(totalWeight) + "%):"] = totalScoresList
    else:
        for sectionScore in sectionScores:
            scoreEarned = scoreEarned + sectionScore
        for maxScore in sectionMaxScores:
            maxScoreCanReceive = maxScoreCanReceive + maxScore
        totalScoresList.append("Total Score: " + str(round((scoreEarned / maxScoreCanReceive) * 100, 2)) + "%")
        recipientsGrades["Total:"] = totalScoresList
    
    df = pd.DataFrame(recipientsGrades)
    return df

In [8]:
file_to_read = "Grades.csv"
file_to_write = "Grades-post-processed.csv"
gradesDF = pd.read_csv(file_to_read)
grades = createDictionary(gradesDF)

df = calculate(grades, gradesDF)
if isinstance(df, str):
    print(df)
else:
    df.to_csv(file_to_write, index = False)
    print(df)

   Discussion (Weight: 7%):  Participation (Weight: 4%):  \
0                       NaN                          NaN   
1                       NaN                          NaN   
2                       NaN                          NaN   
3                       NaN                          NaN   
4                       NaN                          NaN   
5                       NaN                          NaN   
6                       NaN                          NaN   
7                       NaN                          NaN   
8            10/10 | 100.0%             100/100 | 100.0%   
9  Discussion Score: 100.0%  Participation Score: 100.0%   

  Worksheets (Weight: 15%): Exams (Weight: 25%): SA Writing (Weight: 24%):  \
0              5/5 | 100.0%                  NaN                       NaN   
1            4.68/5 | 93.6%                  NaN                       NaN   
2              5/5 | 100.0%                  NaN                       NaN   
3              5/5 | 100.0%